Define variables and import required function.

In [25]:
from math import log
from tqdm import tqdm

#Define the call outcomes for t<25 and t>25
CALL_OUTCOMES = {("LINE BUSY", 0.2, 0.2, 6+3+1), ("UNAVAILABLE", 0.3, 0.5, 6+25+1), ("AVAILABLE", 0.5, 1.0, 6+1)}
PRECISION = 5

Create a random number generator using linear congruential random number algorithm.

In [3]:
def random_number_gen(x0=1000, a=24693, c=3517, k=(2**17)):
    while True:
        x_i = (a * x0 + c) % k
        yield (x_i/k)
        x0 = x_i

Create random variable generators for both the call outcome $C$ and total call time $X$ to simulate call.

In [6]:
def determine_call_time(random_num): #previous cont_rand_var_gen
    return -12*log(1-random_num)

def simulate_call(random_num):
    # determine call outcome by iterating through all outcomes and finding the first match to CDF
    for outcome in CALL_OUTCOMES:
        if outcome[2] >= random_num:
            call_outcome = outcome
            if call_outcome[0] == "AVAILABLE":
                time_waited = determine_call_time((random_num - 0.5) / 0.5)
                if time_waited <= 25:
                    return ("AVAILABLE", 0.5, 1.0, round(6 + time_waited + 1, PRECISION))
                else:
                    return ("UNAVAILABLE", 0.5, 1.0, 32)
            else:
                return ("UNAVAILABLE", 0.5, 1.0, 32)

Run the simulation.

In [30]:
total_customers_called = 0
total_time_spent = 0
call_time_t = []
total_time_w = []
generator = random_number_gen()

number_of_calls_on_customer = 0
current_time = 0

for call_index in tqdm(range(10000)):
    rand_num = next(generator)
    call_outcome, _, _, time_spent_t = simulate_call(rand_num)
    number_of_calls_on_customer += 1 
    call_time_t.append(time_spent_t)
    current_time += time_spent_t

    if call_outcome == "AVAILABLE":
        total_time_w.append(current_time)
        current_time = 0
        number_of_calls_on_customer = 0

    elif number_of_calls_on_customer == 4:
            total_time_w.append(current_time)
            current_time = 0
            number_of_calls_on_customer = 0     

100%|██████████| 10000/10000 [00:00<00:00, 1050545.77it/s]


Some statisics.

In [31]:
total_time_w
print (f"Average time per customer : {round(sum(total_time_w)/len(total_time_w), PRECISION)} seconds")
print (f"Number of customers : {len(total_time_w)}")
print (f"Average call time : {round(sum(call_time_t)/len(call_time_t), PRECISION)} seconds")
print (f"Average number of calls per customer : {round(len(call_time_t)/len(total_time_w), PRECISION)}")

Average time per customer : 51.33777 seconds
Number of customers : 4840
Average call time : 24.84748 seconds
Average number of calls per customer : 2.06612
